In [1]:
import rdflib
import numpy as np
from SPARQLWrapper import SPARQLWrapper, JSON

# Load Word Embeddings

In [2]:
import logging
from six import iteritems
from web.embeddings import load_embedding, fetch_conceptnet_numberbatch
from web.evaluate import evaluate_similarity, evaluate_similarity_score

In [3]:
%%time
#Fetch FastText-wiki 300d 1M
we =load_embedding("C:\\Users\\daix8\\web_data\\embeddings\\fastText\\wiki-news-300d-1M.vec\\wiki-news-300d-1M.vec", format='word2vec', normalize=True, lower=False, clean_words=False)

Wall time: 3min 48s


In [3]:
%%time
#Fetch ConceptNetNumb
wecn = fetch_conceptnet_numberbatch()

File already downloaded, skipping
Wall time: 1min 49s


In [4]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

# isEntityInDatabase Method

In [5]:
def isEntityInDatabase(s):
    return s.startswith("http")

# MakeQueryString Method

In [6]:
def makeQueryString(obj, filterStringsPredicates, filterStringSubjects, literals):
    if obj in literals:
        result = "SELECT distinct ?s ?p WHERE { " + "{?s ?p " + obj + " . " + "} FILTER ("
        for s in filterStringsPredicates:
            fsp = "!regex(str(?p), '" + s + "' , 'i') && "
            result += fsp
        for s in filterStringSubjects:
            fss = "!regex(str(?s), '" + s + "' , 'i') && "
            result += fss
        result = result[:result.rindex("&&")] + ")}"
    else:
        result = "SELECT distinct ?s ?p ?j ?k WHERE { "
        incomingQuery = "{?s ?p <" + obj + "> . FILTER ("
        outgoingQuery =  "{<" + obj + "> ?j ?k. FILTER ("
        
        for s in filterStringsPredicates:
            fsp = "!regex(str(?p), '" + s + "' , 'i') && "
            incomingQuery += fsp
            fsj = "!regex(str(?j), '" + s + "' , 'i') && "
            outgoingQuery += fsj
            
        for s in filterStringSubjects:
            fss = "!regex(str(?s), '" + s + "' , 'i') && "
            fss2 = "!regex(str(?k), '" + s + "' , 'i') && "
            incomingQuery += fss
            outgoingQuery += fss2
        
        incomingQuery = incomingQuery[:incomingQuery.rindex("&&")] + ")}"
        outgoingQuery = outgoingQuery[:outgoingQuery.rindex("&&")] + ")}"
        result = result + incomingQuery + " UNION " + outgoingQuery + "}"
    return result

# Triple Class 

In [7]:
class Triple:
    def __init__(self):
        self.subject = None
        self.object = None
        self.predicate = None
        self.cost = None
        self.previousTriple = None
        #the list of connecting seeds
        self.seeds = []
    
    def __str__(self):
        if(self.subject is not None and self.predicate is not None and self.object is not None):
            # print connecting seeds
            connectSeeds = "["
            for seed in self.seeds:
                connectSeeds += seed + ", "
            connectSeeds = connectSeeds[:connectSeeds.rindex(',')] + "]"
            return self.subject + "  --  " + self.predicate + "  --  " + self.object + "  " + connectSeeds + " (" + str(self.cost) + ')'
        else:
            return "Not Well Defined Triple"
        
    def __eq__(self,other):
        if(self.subject == other.getSubject() and self.object == other.getObject() and self.predicate == other.getPredicate()):
            return True
        else:
            return False
    
    def setSeeds(self, fs):
        if fs not in self.seeds:
            self.seeds.append(fs)
            self.seeds.sort()
            
    def getSeeds(self):
        return self.seeds
    
    def setPreviousTriple(self, pt):
        self.previousTriple = pt
        
    def getPreviousTriple(self):
        return self.previousTriple
    
    def updateSeeds(self, newSeeds):
        for seed in newSeeds:
            self.setSeeds(seed)
            
    def setSubject(self, sub):
        self.subject = sub
    def setObject(self, obj):
        self.object = obj
    def setPredicate(self, pre):
        self.predicate = pre
    def setCost(self, c):
        self.cost = c
    def getCost(self):
        return self.cost
    def getSubject(self):
        return self.subject
    def getObject(self):
        return self.object
    def getPredicate(self):
        return self.predicate

# Expand Method

In [53]:
'''
def expand(queryTriples, eg, di, predicatesToMatch, wemb):
    newQueryTriples = []
    matchingTriples = []
    
    tmp = []
    print("queryTriple size : " + str(len(queryTriples)))
    for queryObject in queryTriples:
        queryString = makeQueryString(queryObject.getObject(), filterStringPredicates,filterStringSubjects, literalsToConnect)
        print("----- Current Query ------")
        print(queryString)
        print("--------------------------")
        
        sparql.setQuery(queryString)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        for result in results["results"]["bindings"]:
            if 's' in result:
                if(isEntityInDatabase(result["s"]["value"])):
                    #print(result["s"]["value"] + "   ---   " + result["p"]["value"])
                    newTriple = Triple()
                    newTriple.setSubject(queryObject.getObject())
                    newTriple.setPredicate(result["p"]["value"])
                    newTriple.setObject(result["s"]["value"])
                    newTriple.setCost(computeCosineSimilarity(predicatesToMatch, result["p"]["value"], wemb ))
                    #newTriple.setCost(20)
                    newTriple.setPreviousTriple(queryObject)
                    
                    for seed in queryObject.getSeeds():
                        newTriple.setSeeds(seed)
                    print(newTriple)
                    
                    if newTriple.getCost() < 50:
                        matchingTriples.append(newTriple)
                    """
                    #TODO: change this to word embedding
                    for p in predicatesToMatch:
                        if p in result["p"]["value"] and newTriple not in matchingTriples:
                            newTriple.setCost(10)
                            matchingTriples.append(newTriple)
                    """
                    if newTriple not in expandedGraph:
                        expandedGraph.append(newTriple)
                    if newTriple.getObject() not in tmp:
                        newQueryTriples.append(newTriple)
                    
                    #Check if it's in tmp, if yes, add triple into duplicatedItems
                    if newTriple.getObject() in tmp:
                        print("add duplicated Items: " + newTriple.getObject())
                        addDuplicatedItems(newTriple, eg, di)
                    else:
                        tmp.append(newTriple.getObject())
            else:
                if(isEntityInDatabase(result["k"]["value"])):
                    #print(result["k"]["value"] + "   ---   " + result["j"]["value"])
                    
                    newTriple = Triple()
                    newTriple.setSubject(queryObject.getObject())
                    newTriple.setPredicate(result["j"]["value"])                
                    newTriple.setObject(result["k"]["value"])
                    #newTriple.setCost(20)
                    newTriple.setCost(computeCosineSimilarity(predicatesToMatch, result["j"]["value"], wemb ))
                    newTriple.setPreviousTriple(queryObject)
                    
                    for seed in queryObject.getSeeds():
                        newTriple.setSeeds(seed)
                    
                    print(newTriple)
                    
                    if newTriple.getCost() < 50:
                        matchingTriples.append(newTriple)
                    """
                    #TODO: change this to word embedding
                    for p in predicatesToMatch:
                        if p in result["j"]["value"] and newTriple not in matchingTriples:
                            newTriple.setCost(10)
                            matchingTriples.append(newTriple)
                    """
                    
                    if newTriple not in expandedGraph:
                        expandedGraph.append(newTriple)
                    if newTriple.getObject() not in tmp:
                        newQueryTriples.append(newTriple)
                        
                    #Check if it's in tmp, if yes, add triple into duplicatedItems
                    if newTriple.getObject() in tmp:
                        print("add duplicated Items: " + newTriple.getObject())
                        addDuplicatedItems(newTriple, eg, di)
                    else:
                        tmp.append(newTriple.getObject())
                        
    return newQueryTriples, matchingTriples
'''
print("Old version of expand")

Old version of expand


In [8]:
def expand(queryTriples, eg, di, predicatesToMatch, wemb):
    newQueryTriples = []
    matchingTriples = []
    
    tmp = []
    print("queryTriple size : " + str(len(queryTriples)))
    for queryObject in queryTriples:
        queryString = makeQueryString(queryObject.getObject(), filterStringPredicates,filterStringSubjects, literalsToConnect)
        print("----- Current Query ------")
        print(queryString)
        print("--------------------------")
        
        sparql.setQuery(queryString)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        for result in results["results"]["bindings"]:
            if 's' in result:
                if(isEntityInDatabase(result["s"]["value"])):
                    #print(result["s"]["value"] + "   ---   " + result["p"]["value"])
                    newTriple = Triple()
                    newTriple.setSubject(queryObject.getObject())
                    newTriple.setPredicate(result["p"]["value"])
                    newTriple.setObject(result["s"]["value"])
                    #newTriple.setCost(computeCosineSimilarity(predicatesToMatch, result["p"]["value"], wemb ))
                    #newTriple.setCost(20)
                    newTriple.setPreviousTriple(queryObject)
                    
                    for seed in queryObject.getSeeds():
                        newTriple.setSeeds(seed)
                    #print(newTriple)
                    
                    #Apply word embedding for costs and modify matchingTriples
                    if newTriple not in matchingTriples:
                        matchingTriples.append(newTriple)
                    
                    if newTriple not in expandedGraph:
                        expandedGraph.append(newTriple)
                    if newTriple.getObject() not in tmp:
                        newQueryTriples.append(newTriple)
                    
                    #Check if it's in tmp, if yes, add triple into duplicatedItems
                    if newTriple.getObject() in tmp:
                        print("add duplicated Items: " + newTriple.getObject())
                        addDuplicatedItems(newTriple, eg, di)
                    else:
                        tmp.append(newTriple.getObject())
            else:
                if(isEntityInDatabase(result["k"]["value"])):
                    #print(result["k"]["value"] + "   ---   " + result["j"]["value"])
                    
                    newTriple = Triple()
                    newTriple.setSubject(queryObject.getObject())
                    newTriple.setPredicate(result["j"]["value"])                
                    newTriple.setObject(result["k"]["value"])
                    #newTriple.setCost(20)
                    #newTriple.setCost(computeCosineSimilarity(predicatesToMatch, result["j"]["value"], wemb ))
                    newTriple.setPreviousTriple(queryObject)
                    
                    for seed in queryObject.getSeeds():
                        newTriple.setSeeds(seed)
                    
                    #print(newTriple)
                    
                    #Apply word embedding for costs and modify matchingTriples
                    if newTriple not in matchingTriples:
                        matchingTriples.append(newTriple)
                    
                    if newTriple not in expandedGraph:
                        expandedGraph.append(newTriple)
                    if newTriple.getObject() not in tmp:
                        newQueryTriples.append(newTriple)
                        
                    #Check if it's in tmp, if yes, add triple into duplicatedItems
                    if newTriple.getObject() in tmp:
                        print("add duplicated Items: " + newTriple.getObject())
                        addDuplicatedItems(newTriple, eg, di)
                    else:
                        tmp.append(newTriple.getObject())
        computeCosts(predicatesToMatch, matchingTriples, wemb)
    return newQueryTriples, matchingTriples

# addDuplicatedItems Method

In [9]:
def addDuplicatedItems(ntp, eg, di):
    for tp in eg:
        if tp.getObject() == ntp.getObject() and tp not in di:
            di.append(tp)
            if tp.getSeeds() != ntp.getSeeds():
                updateSeedsInExpandedGraph(tp,ntp,eg)           
    #di.append(ntp)
    return "Dup"

In [10]:
def updateSeedsInExpandedGraph(tp,ntp,eg):
    old1 = tp.getSeeds()[:]
    old2 = ntp.getSeeds()[:]
    ntp.updateSeeds(tp.getSeeds())
    newS = ntp.getSeeds()
    
    for triple in eg:
        if triple.getSeeds() == old1 or triple.getSeeds() == old2:
            triple.updateSeeds(newS)

# Reduction Tests

In [11]:
def reductionTestsDegreeOne(eg):
    
    degreeOneNodes = []
    
    for tp1 in eg:
        duplicatedObject = False
        isLeafNode = True
        for tp2 in eg:
            if tp2.getPreviousTriple() == tp1:
                isLeafNode = False
            if tp1.getObject() == tp2.getObject() and not tp1.getSubject() == tp2.getSubject():
                duplicatedObject = True
                break
        if not duplicatedObject and isLeafNode:
            degreeOneNodes.append(tp1)
    print("Degree One Nodes size: " + str(len(degreeOneNodes)))
    for tp in degreeOneNodes:
        eg.remove(tp)

#TODO: Fix Bug
def keepMinEdge(eg):
    
    triplesToBeRemoved = []
    
    for tp1 in eg:
        minTriple = tp1
        for tp2 in eg:
            if tp1.getSubject() == tp2.getSubject() and tp1.getObject() == tp2.getObject and not tp1.getPredicate() == tp2.getPredicate():
                if tp2.getCost() < tp1.getCost():
                    minTriple = tp2
                    if tp1 not in triplesToBeRemoved:
                        triplesToBeRemoved.append(tp1)
                else:
                    if tp2 not in triplesToBeRemoved:
                        triplesToBeRemoved.append(tp2)
    for tp in triplesToBeRemoved:
        eg.remove(tp)
    

# checkConnection Method

In [12]:
def checkConnection(ltc, di):
    if len(ltc) == 1:
        return True
    else:
        ltc.sort()
        for tp in di:
            if tp.getSeeds() == ltc:
                return True
        return False

# ComputeCosts Method

In [30]:
def computeCosts(predicatesToMatch, matchingTriples, wemb):
    th = 50
    predicateList = []
    mt = []
    for tp in matchingTriples:
        predicate = tp.getPredicate()
        predicateList.append(predicate[predicate.rindex('/')+1:])
        
    for x in predicatesToMatch:
        ar = []
        for p in predicateList:
            ar.append([x, p])
        ar = np.array(ar)
        result = evaluate_similarity_score(wemb, ar, 'xxx')
        for x in range(len(result)):
            if matchingTriples[x].getCost() is None:
                matchingTriples[x].setCost((1 - result[x]) * 100)
            elif matchingTriples[x].getCost() > (1 - result[x]) * 100:
                matchingTriples[x].setCost((1 - result[x]) * 100)
    
    index = 0
    
    while index < len(matchingTriples):
        if matchingTriples[index].getCost() > th:
            matchingTriples.pop(index)
        else:
            index += 1


# Main Program

In [37]:
#literalsToConnect = ['"Viking Press"@en','"Jack Kerouac"@en']
#literalsToConnect = ['"Barack Obama"@en','"Michelle Obama"@en']
#literalsToConnect = ['"Rita Wilson"@en','"Tom Hanks"@en']
literalsToConnect = ['"Berlin"@en']
filterStringPredicates = ["wikiPageWikiLink","wikiPageRedirects","wikiPageDisambiguates", "Thing","wikiPageUsesTemplate","rdf-syntax-ns#type"]
filterStringSubjects = ["entity", "Category", "wikidata","owl#Thing", "http://wikidata.dbpedia.org/resource/Q"]
expandedGraph = []
duplicatedItems = []
predicates = ['leader']

In [38]:
qtps = []
for literal in literalsToConnect:
    triple = Triple()
    triple.setObject(literal)
    triple.setSeeds(literal)
    qtps.append(triple)
    
for tp in qtps:
    print(tp.getObject())

"Berlin"@en


In [39]:
%%time
levelOfExpansion = 1
while(levelOfExpansion < 3 or not checkConnection(literalsToConnect, duplicatedItems)):
    #add mtps to a new list
    qtps, mtps = expand(qtps, expandedGraph, duplicatedItems, predicates, we)
    levelOfExpansion += 1

print("Matching Triples: -----   size: " + str(len(mtps)))
for tp in mtps:
    print(tp)

queryTriple size : 1
----- Current Query ------
SELECT distinct ?s ?p WHERE { {?s ?p "Berlin"@en . } FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )}
--------------------------


Missing 3 words. Will replace them with mean vector


queryTriple size : 59
----- Current Query ------
SELECT distinct ?s ?p ?j ?k WHERE { {?s ?p <http://dbpedia.org/resource/Berlin> . FILTER (!regex(str(?p), 'wikiPageWikiLink' , 'i') && !regex(str(?p), 'wikiPageRedirects' , 'i') && !regex(str(?p), 'wikiPageDisambiguates' , 'i') && !regex(str(?p), 'Thing' , 'i') && !regex(str(?p), 'wikiPageUsesTemplate' , 'i') && !regex(str(?p), 'rdf-syntax-ns#type' , 'i') && !regex(str(?s), 'entity' , 'i') && !regex(str(?s), 'Category' , 'i') && !regex(str(?s), 'wikidata' , 'i') && !regex(str(?s), 'owl#Thing' , 'i') && !regex(str(?s), 'http://wikidata.dbpedia.org/resource/Q' , 'i') )} UNION {<http://dbpedia.org/resource/Berlin> ?j ?k. FILTER (!regex(str(?j), 'wikiPageWikiLink' , 'i') && !regex(str(?j), 'wikiPageRedirects' , 'i') && !regex(str(?j), 'wikiPageDisambiguates' , 'i') && !regex(str(?j), 'Thing' , 'i') && !regex(str(?j), 'wikiPageUsesTemplate' , 'i') && !regex(str(?j), 'rdf-syntax-ns#type' , 'i') && !regex(str(?k), 'entity' , 'i') && !regex(str(

add duplicated Items: http://dbpedia.org/resource/Hans-Thilo_Schmidt
add duplicated Items: http://dbpedia.org/resource/Friedrich_Wilhelm_Jähns
add duplicated Items: http://dbpedia.org/resource/Gustav_Richter_(artist)
add duplicated Items: http://dbpedia.org/resource/Prince_Louis_Charles_of_Prussia
add duplicated Items: http://dbpedia.org/resource/August_Immanuel_Bekker
add duplicated Items: http://dbpedia.org/resource/Alfred_von_Schlieffen
add duplicated Items: http://dbpedia.org/resource/Eduard_Gans
add duplicated Items: http://dbpedia.org/resource/Friedrich_Eduard_Beneke
add duplicated Items: http://dbpedia.org/resource/Horst_Buchholz
add duplicated Items: http://dbpedia.org/resource/Ludwig_Tieck
add duplicated Items: http://dbpedia.org/resource/Walther_Rathenau
add duplicated Items: http://dbpedia.org/resource/Carl_Friedrich_Heinrich,_Graf_von_Wylich_und_Lottum
add duplicated Items: http://dbpedia.org/resource/Otto_Winzer
add duplicated Items: http://dbpedia.org/resource/Wilhelm_Sol

add duplicated Items: http://dbpedia.org/resource/Harry_Frank
add duplicated Items: http://dbpedia.org/resource/Heinrich_Roller
add duplicated Items: http://dbpedia.org/resource/Hermine_Körner
add duplicated Items: http://dbpedia.org/resource/Horst_Wendlandt
add duplicated Items: http://dbpedia.org/resource/Jakob_Heinrich_Laspeyres
add duplicated Items: http://dbpedia.org/resource/Johann_Friedrich_Zückert
add duplicated Items: http://dbpedia.org/resource/Karl_Bolle
add duplicated Items: http://dbpedia.org/resource/Leo_Arons
add duplicated Items: http://dbpedia.org/resource/Magnus_von_Eberhardt
add duplicated Items: http://dbpedia.org/resource/Marion_Yorck_von_Wartenburg
add duplicated Items: http://dbpedia.org/resource/Richard_Kolkwitz
add duplicated Items: http://dbpedia.org/resource/Samuel_von_Schmettau
add duplicated Items: http://dbpedia.org/resource/Wolf_Jobst_Siedler
add duplicated Items: http://dbpedia.org/resource/Wolfgang_Lukschy
add duplicated Items: http://dbpedia.org/resour

...

Matching Triples: -----   size: 223
http://dbpedia.org/resource/Berlin_(The_Blacklist)  --  http://dbpedia.org/ontology/director  --  http://dbpedia.org/resource/Michael_Zinberg  ["Berlin"@en] (46.006232500076294)
http://dbpedia.org/resource/Berlin_(The_Blacklist)  --  http://dbpedia.org/ontology/writer  --  http://dbpedia.org/resource/John_Eisendrath  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Berlin_(The_Blacklist)  --  http://dbpedia.org/ontology/writer  --  http://dbpedia.org/resource/Jon_Bokenkamp  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Berlin_(band)  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/The_Teaches_of_Peaches  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Berlin:_Live_at_St._Ann's_Warehouse  --  http://dbpedia.org/ontology/director  --  http://dbpedia.org/resource/Julian_Schnabel  ["Berlin"@en] (46.006232500076294)
http://dbpedia.org/resource/Berlin_Ndebe-Nlome  --  http://dbpedia.org/ont

http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/Off_White  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/Any_Given_Sunday_(soundtrack)  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/Ballad_Essentials  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/She_Was_Too_Good_to_Me  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/The_Very_Best_of_Al_Jolson  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/Young_Man_with_a_Horn_(Mile

http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/Glory_in_the_Highest:_A_Christmas_Record  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/Merry_Christmas_to_You_(Sidewalk_Prophets_album)  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/Stages:_Performances_1970–2002  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/Willie’s_Stash,_Vol._1:_December_Day  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/Buon_Natale:_The_Christmas_Album  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbped

http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/Postcard_(album)  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/Otis!_The_Definitive_Otis_Redding  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/Everything_I_Love_(Eliane_Elias_album)  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/Holiday_Wishes_(Idina_Menzel_album)  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/A_Faithful_Christmas  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.o

http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/This_Christmas_(John_Travolta_and_Olivia_Newton-John_album)  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/Ultimate_Christmas  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/Merry_Christmas_Strait_to_You!  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/The_Soulful_Moods_of_Marvin_Gaye  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  http://dbpedia.org/resource/Coltrane_for_Lovers  ["Berlin"@en] (49.59213137626648)
http://dbpedia.org/resource/Irving_Berlin  --  http://dbpedia.org/property/writer  --  

In [18]:
print("Size of expanded graph: "+ str(len(expandedGraph)))
for tp in expandedGraph:
    print(tp)

Size of expanded graph: 606
"Viking Press"@en  --  http://www.w3.org/2000/01/rdf-schema#label  --  http://dbpedia.org/resource/Viking_Press  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
"Jack Kerouac"@en  --  http://www.w3.org/2000/01/rdf-schema#label  --  http://dbpedia.org/resource/Jack_Kerouac  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
"Jack Kerouac"@en  --  http://xmlns.com/foaf/0.1/name  --  http://dbpedia.org/resource/Jack_Kerouac  ["Jack Kerouac"@en, "Viking Press"@en] (64.26130831241608)
http://dbpedia.org/resource/Viking_Press  --  http://xmlns.com/foaf/0.1/primaryTopic  --  http://en.wikipedia.org/wiki/Viking_Press  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Grasshopper_(novel)  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publ

http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/My_Year_of_Meats  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/The_Fort_at_River's_Bend  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Kingdoms_of_Elfin  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/The_Multiple_Effects_of_Rainshadow  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/The_Diddakoi  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
ht

http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/The_Third_Mind  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Dear_Zoe  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Death_from_the_Skies!  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Lost_in_the_Garden  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/The_Pale_Criminal  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource

http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Spycatcher  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/The_Mist  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Dolan's_Cadillac_(short_story)  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Moon_Palace  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/A_Tiger_for_Malgudi  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource

http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Being_Dead  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Miss_Hickory  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/The_Blood_Doctor  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/The_Chimney-sweeper's_Boy  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/The_House_of_Stairs_(Vine_novel)  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbp

...

http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/The_Finders  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Night_of_the_Aurochs  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/In_the_Country_of_Last_Things  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Asta's_Book  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Europe_Central  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/r

http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/property/founders  --  http://dbpedia.org/resource/George_S._Oppenheim  ["Jack Kerouac"@en, "Viking Press"@en] (71.60378098487854)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/property/founders  --  http://dbpedia.org/resource/Harold_K._Guinzburg  ["Jack Kerouac"@en, "Viking Press"@en] (71.60378098487854)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/property/headquarters  --  http://dbpedia.org/resource/New_York_City  ["Jack Kerouac"@en, "Viking Press"@en] (71.63157761096954)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/property/headquarters  --  http://dbpedia.org/resource/New_York_(state)  ["Jack Kerouac"@en, "Viking Press"@en] (71.63157761096954)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/property/imprints  --  http://dbpedia.org/resource/Pamela_Dorman  ["Jack Kerouac"@en, "Viking Press"@en] (68.67101788520813)
http://dbpedia.org/resource

http://dbpedia.org/resource/Jack_Kerouac  --  http://dbpedia.org/ontology/influenced  --  http://dbpedia.org/resource/William_S._Burroughs  ["Jack Kerouac"@en, "Viking Press"@en] (63.84537220001221)
http://dbpedia.org/resource/Jack_Kerouac  --  http://dbpedia.org/ontology/influenced  --  http://dbpedia.org/resource/William_Gaddis  ["Jack Kerouac"@en, "Viking Press"@en] (63.84537220001221)
http://dbpedia.org/resource/Jack_Kerouac  --  http://dbpedia.org/ontology/influenced  --  http://dbpedia.org/resource/Mark_Van_Doren  ["Jack Kerouac"@en, "Viking Press"@en] (63.84537220001221)
http://dbpedia.org/resource/Jack_Kerouac  --  http://dbpedia.org/ontology/influenced  --  http://dbpedia.org/resource/Gary_Snyder  ["Jack Kerouac"@en, "Viking Press"@en] (63.84537220001221)
http://dbpedia.org/resource/Jack_Kerouac  --  http://dbpedia.org/ontology/influenced  --  http://dbpedia.org/resource/Louis-Ferdinand_Céline  ["Jack Kerouac"@en, "Viking Press"@en] (63.84537220001221)
http://dbpedia.org/resou

http://dbpedia.org/resource/Jack_Kerouac  --  http://www.w3.org/2002/07/owl#sameAs  --  http://zitgist.com/music/artist/ccf81e74-b701-4277-a599-3e0942e70b53  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
http://dbpedia.org/resource/Jack_Kerouac  --  http://www.w3.org/2002/07/owl#sameAs  --  http://sw.cyc.com/concept/Mx4rvcQ_ZJwpEbGdrcN5Y29ycA  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
http://dbpedia.org/resource/Jack_Kerouac  --  http://www.w3.org/2002/07/owl#sameAs  --  http://yago-knowledge.org/resource/Jack_Kerouac  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
http://dbpedia.org/resource/Jack_Kerouac  --  http://www.w3.org/2002/07/owl#sameAs  --  http://rdf.freebase.com/ns/m.041mt  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
http://dbpedia.org/resource/Jack_Kerouac  --  http://www.w3.org/2002/07/owl#sameAs  --  http://d-nb.info/gnd/118561561  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
http://dbpedia.org/resou

http://dbpedia.org/resource/Jack_Kerouac  --  http://dbpedia.org/ontology/influenced  --  http://dbpedia.org/resource/Blake_Schwarzenbach  ["Jack Kerouac"@en, "Viking Press"@en] (63.84537220001221)
http://dbpedia.org/resource/Jack_Kerouac  --  http://dbpedia.org/ontology/influenced  --  http://dbpedia.org/resource/Will_Clarke_(novelist)  ["Jack Kerouac"@en, "Viking Press"@en] (63.84537220001221)
http://dbpedia.org/resource/Jack_Kerouac  --  http://dbpedia.org/ontology/influenced  --  http://dbpedia.org/resource/D.M._Court  ["Jack Kerouac"@en, "Viking Press"@en] (63.84537220001221)
http://dbpedia.org/resource/Jack_Kerouac  --  http://dbpedia.org/ontology/influencedBy  --  http://dbpedia.org/resource/Charlie_Parker  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
http://dbpedia.org/resource/Jack_Kerouac  --  http://dbpedia.org/ontology/influencedBy  --  http://dbpedia.org/resource/Honoré_de_Balzac  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
http://dbpedia.org/r

In [19]:
print("Size of duplicatedItems: "+ str(len(duplicatedItems)))
for tp in duplicatedItems:
    print(tp)

Size of duplicatedItems: 39
"Jack Kerouac"@en  --  http://www.w3.org/2000/01/rdf-schema#label  --  http://dbpedia.org/resource/Jack_Kerouac  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
"Jack Kerouac"@en  --  http://xmlns.com/foaf/0.1/name  --  http://dbpedia.org/resource/Jack_Kerouac  ["Jack Kerouac"@en, "Viking Press"@en] (64.26130831241608)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/wikiPageExternalLink  --  http://penguin.com  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
http://dbpedia.org/resource/Viking_Press  --  http://xmlns.com/foaf/0.1/homepage  --  http://penguin.com  ["Jack Kerouac"@en, "Viking Press"@en] (67.68727004528046)
http://dbpedia.org/resource/Viking_Press  --  http://xmlns.com/foaf/0.1/primaryTopic  --  http://en.wikipedia.org/wiki/Viking_Press  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
http://dbpedia.org/resource/Viking_Press  --  http://xmlns.com/foaf/0.1/isPrimaryTopicOf  --  http://en.

In [20]:
reductionTestsDegreeOne(expandedGraph)
print("Size of expanded graph: "+ str(len(expandedGraph)))
for tp in expandedGraph:
    print(tp)

Degree One Nodes size: 597
Size of expanded graph: 9
"Viking Press"@en  --  http://www.w3.org/2000/01/rdf-schema#label  --  http://dbpedia.org/resource/Viking_Press  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
"Jack Kerouac"@en  --  http://www.w3.org/2000/01/rdf-schema#label  --  http://dbpedia.org/resource/Jack_Kerouac  ["Jack Kerouac"@en, "Viking Press"@en] (64.37246203422546)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/On_the_Road  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Atop_an_Underwood:_Early_Stories_and_Other_Writings  ["Jack Kerouac"@en, "Viking Press"@en] (51.287105679512024)
http://dbpedia.org/resource/Viking_Press  --  http://dbpedia.org/ontology/publisher  --  http://dbpedia.org/resource/Door_Wide_Open  ["Jack Kerouac"@en, "Viking Press"@en] (51.2871056

In [20]:
keepMinEdge(expandedGraph)
print("Size of expanded graph: "+ str(len(expandedGraph)))
for tp in expandedGraph:
    print(tp)

Size of expanded graph: 0


# Test Purpose -- Main

In [ ]:
def isResourceInDataset():
    